In [3]:
from IPython.display import display
import pandas as pd


# Load CSVs
influencers = pd.read_csv("data/influencers.csv")
posts = pd.read_csv("data/posts.csv")
tracking = pd.read_csv("data/tracking_data.csv")
payouts = pd.read_csv("data/payouts.csv")

# Preview
print("Influencers:", influencers.shape)
display(influencers.head())

print("Posts:", posts.shape)
display(posts.head())

print("Tracking Data:", tracking.shape)
display(tracking.head())

print("Payouts:", payouts.shape)
display(payouts.head())

# Check for missing values
print("\nMissing Values:")
print("Influencers:\n", influencers.isnull().sum())
print("Posts:\n", posts.isnull().sum())
print("Tracking:\n", tracking.isnull().sum())
print("Payouts:\n", payouts.isnull().sum())


Influencers: (5, 6)


,influencer_id,name,category,gender,follower_count,platform
0,1,RiyaFitness,Fitness,Female,500000,Instagram
1,2,TechWithRaj,Tech,Male,200000,YouTube
2,3,GlowByTina,Beauty,Female,300000,Instagram
3,4,FitGuru,Fitness,Male,800000,Twitter
4,5,LifeHacks101,Lifestyle,Other,150000,YouTube


Posts: (5, 9)


,influencer_id,platform,date,URL,caption,reach,likes,comments,brand
0,1,Instagram,01-06-2024,url1.com/post1,Summer Workout Plan,450000,15000,1200,MuscleBlaze
1,2,YouTube,02-06-2024,url2.com/video1,Gadget Review,180000,8000,700,TrueBasics
2,3,Instagram,03-06-2024,url3.com/post2,Skincare Routine,250000,10000,1100,HK Vitals
3,4,Twitter,04-06-2024,url4.com/tweet1,Protein Facts,700000,9000,600,bGreen
4,5,YouTube,05-06-2024,url5.com/video2,Life Hack Tuesday,120000,3000,200,Gritzo


Tracking Data: (5, 8)


,source,campaign,influencer_id,user_id,product,date,orders,revenue
0,Instagram,MB_Summer2024,1,u101,MuscleBlaze Whey,2024-06-02,20,10000
1,YouTube,Tech_Launch,2,u102,HKVitals Multis,2024-06-03,10,5000
2,Instagram,Glow_Skin24,3,u103,Gritzo Kids Drink,2024-06-04,15,7500
3,Twitter,MB_ProSeries,4,u104,MB Protein Bar,2024-06-05,25,12500
4,YouTube,Hacks_Week1,5,u105,HK Shaker Bottle,2024-06-06,5,2500


Payouts: (5, 5)


,influencer_id,basis,rate,orders,total_payout
0,1,post,5000,20,5000
1,2,order,200,10,2000
2,3,post,4000,15,4000
3,4,order,300,25,7500
4,5,order,300,5,1500



Missing Values:
Influencers:
 influencer_id     0
name              0
category          0
gender            0
follower_count    0
platform          0
dtype: int64
Posts:
 influencer_id    0
platform         0
date             0
URL              0
caption          0
reach            0
likes            0
comments         0
brand            0
dtype: int64
Tracking:
 source           0
campaign         0
influencer_id    0
user_id          0
product          0
date             0
orders           0
revenue          0
dtype: int64
Payouts:
 influencer_id    0
basis            0
rate             0
orders           0
total_payout     0
dtype: int64


In [33]:
# Merge tracking and payouts on influencer_id for ROAS calculations
merged = pd.merge(tracking, payouts, on="influencer_id", how="left")

# Global metrics
total_revenue = merged["revenue"].sum()
total_payout = merged["total_payout"].sum()
overall_roas = round(total_revenue / total_payout, 2) if total_payout else 0

print("Total Revenue: ₹", total_revenue)
print("Total Payout: ₹", total_payout)
print("Overall ROAS:", overall_roas)

# ROAS per influencer
merged["ROAS"] = merged["revenue"] / merged["total_payout"]

# Join influencer name for readability
merged = pd.merge(merged, influencers[["id", "name"]], left_on="influencer_id", right_on="id", how="left")

roas_per_influencer = merged.groupby(["influencer_id", "name"]).agg({
    "revenue": "sum",
    "total_payout": "sum",
    "ROAS": "mean"
}).reset_index()

# Sort by ROAS
roas_per_influencer = roas_per_influencer.sort_values(by="ROAS", ascending=False)
display(roas_per_influencer)


Total Revenue: ₹ 37500
Total Payout: ₹ 20000
Overall ROAS: 1.88


,influencer_id,name,revenue,total_payout,ROAS
1,2,TechWithRaj,5000,2000,2.500000
0,1,RiyaFitness,10000,5000,2.000000
2,3,GlowByTina,7500,4000,1.875000
3,4,FitGuru,12500,7500,1.666667
4,5,LifeHacks101,2500,1500,1.666667


Top 3 Influencers by Revenue


In [34]:
top_3_revenue_influencers = roas_per_influencer.sort_values(by="revenue", ascending=False).head(3)
print("Top 3 Influencers by Revenue:")
display(top_3_revenue_influencers)

Top 3 Influencers by Revenue:


,influencer_id,name,revenue,total_payout,ROAS
3,4,FitGuru,12500,7500,1.666667
0,1,RiyaFitness,10000,5000,2.000000
2,3,GlowByTina,7500,4000,1.875000


 Influencer(s) with Highest ROAS

In [35]:
highest_roas = roas_per_influencer.sort_values(by="ROAS", ascending=False).head(3)
print("Influencer with highest ROAS:")
display(highest_roas)


Influencer with highest ROAS:


,influencer_id,name,revenue,total_payout,ROAS
1,2,TechWithRaj,5000,2000,2.500
0,1,RiyaFitness,10000,5000,2.000
2,3,GlowByTina,7500,4000,1.875


 Influencer(s) with Lowest ROAS

In [36]:
lowest_roas = roas_per_influencer.sort_values(by="ROAS", ascending=True).head(3)
print("Influencer with lowest ROAS:")
display(lowest_roas)

Influencer with lowest ROAS:


,influencer_id,name,revenue,total_payout,ROAS
3,4,FitGuru,12500,7500,1.666667
4,5,LifeHacks101,2500,1500,1.666667
2,3,GlowByTina,7500,4000,1.875000


ROAS Performance by Platform

In [37]:
# Merge tracking and posts to get platform info
tracking_with_platform = pd.merge(tracking, posts[['influencer_id', 'platform']], left_on='influencer_id', right_on='influencer_id', how='left')

# Join payout info
tracking_with_platform = pd.merge(tracking_with_platform, payouts, on='influencer_id', how='left')

# Compute ROAS at platform level
platform_roas = tracking_with_platform.groupby('platform').agg({
    'revenue': 'sum',
    'total_payout': 'sum'
}).reset_index()

platform_roas['ROAS'] = platform_roas['revenue'] / platform_roas['total_payout']
platform_roas = platform_roas.sort_values(by='ROAS', ascending=False)

print("Platform-wise ROAS:")
display(platform_roas)


Platform-wise ROAS:


,platform,revenue,total_payout,ROAS
2,YouTube,7500,3500,2.142857
0,Instagram,17500,9000,1.944444
1,Twitter,12500,7500,1.666667


Brand-Level ROAS

In [38]:
# Merge posts and tracking data to get brand for each revenue
brand_df = pd.merge(tracking, posts[['influencer_id', 'brand']], left_on='influencer_id', right_on='influencer_id', how='left')

# Merge with payouts to get total payout per influencer
brand_df = pd.merge(brand_df, payouts, on='influencer_id', how='left')

# Group by brand
brand_roas = brand_df.groupby('brand').agg({
    'revenue': 'sum',
    'total_payout': 'sum'
}).reset_index()

# Calculate ROAS
brand_roas['ROAS'] = brand_roas['revenue'] / brand_roas['total_payout']
brand_roas = brand_roas.sort_values(by='ROAS', ascending=False)

print("Brand-wise ROAS:")
display(brand_roas)


Brand-wise ROAS:


,brand,revenue,total_payout,ROAS
3,TrueBasics,5000,2000,2.500000
2,MuscleBlaze,10000,5000,2.000000
1,HK Vitals,7500,4000,1.875000
0,Gritzo,2500,1500,1.666667
4,bGreen,12500,7500,1.666667


Influencer-level ROI / ROAS

In [39]:
# Merge all data
influencer_df = pd.merge(tracking, posts[['influencer_id', 'brand']], on='influencer_id', how='left')
influencer_df = pd.merge(influencer_df, payouts, on='influencer_id', how='left')

# Group by influencer
influencer_roas = influencer_df.groupby('influencer_id').agg({
    'revenue': 'sum',
    'total_payout': 'sum',
    'brand': 'first'  # optional: to see which brand they worked with
}).reset_index()

# Calculate ROI / ROAS
influencer_roas['ROAS'] = influencer_roas['revenue'] / influencer_roas['total_payout']
influencer_roas = influencer_roas.sort_values(by='ROAS', ascending=False)

display(influencer_roas)


,influencer_id,revenue,total_payout,brand,ROAS
1,2,5000,2000,TrueBasics,2.500000
0,1,10000,5000,MuscleBlaze,2.000000
2,3,7500,4000,HK Vitals,1.875000
3,4,12500,7500,bGreen,1.666667
4,5,2500,1500,Gritzo,1.666667


Top Performing Platform

In [40]:
# Merge tracking with posts to get platform info
platform_df = pd.merge(tracking, posts[['influencer_id', 'platform']], on='influencer_id', how='left')

# Group by platform
platform_roas = platform_df.groupby('platform')['revenue'].sum().reset_index().sort_values(by='revenue', ascending=False)

display(platform_roas)


,platform,revenue
0,Instagram,17500
1,Twitter,12500
2,YouTube,7500


Reach vs Revenue Analysis

In [41]:
# Merge reach and revenue
reach_rev = pd.merge(tracking, posts[['influencer_id', 'reach']], on='influencer_id', how='left')

# Drop duplicates if needed
reach_rev = reach_rev.drop_duplicates(subset=['influencer_id'])

# Plot or view correlation
correlation = reach_rev['reach'].corr(reach_rev['revenue'])
print(f"Correlation between reach and revenue: {correlation:.2f}")


Correlation between reach and revenue: 0.96
